<a href="https://colab.research.google.com/github/HJKQWE/PDF_table_list_to_exel/blob/main/PDF_table_list_to_exel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from PIL import Image
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
#!pip install paddlepaddle-gpu==2.3.2 cudatoolkit==10.2

In [4]:
!pip install paddlepaddle-gpu==2.3.0.post110 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html

Looking in links: https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html


In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

# pdf2image

## Installation

In [6]:
!pip install pdf2image
!apt-get update
!apt-get install poppler-utils

Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,367 kB]
Fetched 1,589 kB in 3s (531 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree    

## Conversion

In [7]:
from pdf2image import convert_from_path

In [8]:
images = convert_from_path('/content/drive/MyDrive/ГеоКин/Утевское (Таблица 6)-2018.pdf')

In [9]:
!mkdir pages

mkdir: cannot create directory ‘pages’: File exists


In [10]:
for i in range(len(images)):
  images[i].save('pages/page'+str(i)+'.jpg', 'JPEG')

# Layout

## Installation

In [11]:
#!python3 -m pip install paddlepaddle-gpu
!pip install "paddleocr>=2.0.6"
!pip install protobuf==3.20.0
!git clone https://github.com/PaddlePaddle/PaddleOCR.git

fatal: destination path 'PaddleOCR' already exists and is not an empty directory.


In [12]:
!wget https://paddleocr.bj.bcebos.com/whl/layoutparser-0.0.0-py3-none-any.whl
!pip install -U layoutparser-0.0.0-py3-none-any.whl

--2023-07-05 13:07:28--  https://paddleocr.bj.bcebos.com/whl/layoutparser-0.0.0-py3-none-any.whl
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 103.235.46.61, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|103.235.46.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19145360 (18M) [application/octet-stream]
Saving to: ‘layoutparser-0.0.0-py3-none-any.whl.1’

layoutparser-0.0.0- 100%[===================>]  18.26M  4.73MB/s    in 20s     

2023-07-05 13:07:50 (942 KB/s) - ‘layoutparser-0.0.0-py3-none-any.whl.1’ saved [19145360/19145360]

Processing ./layoutparser-0.0.0-py3-none-any.whl
layoutparser is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


## Table Extraction

In [30]:
!pip install layoutparser torchvision && pip install "detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.5#egg=detectron2"

  Cloning https://github.com/facebookresearch/detectron2.git (to revision v0.5) to /tmp/pip-install-38o5payi/detectron2_f0905311a24442f4b4443add0b79bff9
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-install-38o5payi/detectron2_f0905311a24442f4b4443add0b79bff9
  Running command git checkout -q 82a57ce0b70057685962b352535147d9a8118578
  Resolved https://github.com/facebookresearch/detectron2.git to commit 82a57ce0b70057685962b352535147d9a8118578
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.0/131.0 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 15.3 MB/s eta 0:00:0

In [31]:
import cv2
import layoutparser as lp

image = cv2.imread("/content/pages/page0.jpg")

image = image[..., ::-1]

# load model
model = lp.Detectron2LayoutModel(
            config_path ='lp://PubLayNet/faster_rcnn_R_50_FPN_3x/config', # In model catalog
            label_map   = {0: "Table"}, # In model`label_map`
            extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8] # Optional
        )
# detect
layout = model.detect(image)

config.yml?dl=1: 8.19kB [00:01, 7.52kB/s]
model_final.pth?dl=1: 330MB [00:05, 59.0MB/s]                           
  proposal_generator.anchor_generator.cell_anchors.{0, 1, 2, 3, 4}


In [32]:
layout

Layout(_blocks=[TextBlock(block=Rectangle(x_1=545.2136840820312, y_1=240.0938262939453, x_2=1122.7684326171875, y_2=261.4152526855469), text=None, id=None, type=Table, parent=None, next=None, score=0.996035635471344), TextBlock(block=Rectangle(x_1=133.2216796875, y_1=252.5806121826172, x_2=1559.3548583984375, y_2=2140.6650390625), text=None, id=None, type=3, parent=None, next=None, score=0.9835959672927856), TextBlock(block=Rectangle(x_1=255.57586669921875, y_1=2110.55322265625, x_2=915.71728515625, y_2=2134.183837890625), text=None, id=None, type=Table, parent=None, next=None, score=0.8061378002166748)], page_data={})

In [33]:
x_1=0
y_1=0
x_2=0
y_2=0

for l in layout:
  #print(l)
  if l.type == 'Table':
    x_1 = int(l.block.x_1)
    print(l.block.x_1)
    y_1 = int(l.block.y_1)
    x_2 = int(l.block.x_2)
    y_2 = int(l.block.y_2)

    break

545.2136840820312


In [34]:
print(x_1,y_1,x_2,y_2)

545 240 1122 261


In [66]:
im = cv2.imread('/content/pages/page1.jpg')

In [67]:
cv2.imwrite('ext_im.jpg', im[y_1:y_2,x_1:x_2])

True

# Text Detection and Recognition

In [68]:
from paddleocr import PaddleOCR, draw_ocr

In [69]:
ocr = PaddleOCR(lang='ru')
image_path = '/content/ext_im.jpg'
image_cv = cv2.imread(image_path)
image_height = image_cv.shape[0]
image_width = image_cv.shape[1]
output = ocr.ocr(image_path)[0]

[2023/07/05 13:26:39] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/ml/Multilingual_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/cyrillic/cyrillic_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_

In [70]:
print(output)

[]


In [71]:
boxes = [line[0] for line in output]
texts = [line[1][0] for line in output]
probabilities = [line[1][1] for line in output]

In [72]:
image_boxes = image_cv.copy()


In [73]:
for box,text in zip(boxes,texts):
  cv2.rectangle(image_boxes, (int(box[0][0]),int(box[0][1])), (int(box[2][0]),int(box[2][1])),(0,0,255),1)
  cv2.putText(image_boxes, text,(int(box[0][0]),int(box[0][1])),cv2.FONT_HERSHEY_SIMPLEX,1,(222,0,0),1)

In [74]:
cv2.imwrite('detections.jpg', image_boxes)

True

# Reconstruction

## Get Horizontal and Vertical Lines

In [75]:
im = image_cv.copy()

In [76]:
horiz_boxes = []
vert_boxes = []

for box in boxes:
  x_h, x_v = 0,int(box[0][0])
  y_h, y_v = int(box[0][1]),0
  width_h,width_v = image_width, int(box[2][0]-box[0][0])
  height_h,height_v = int(box[2][1]-box[0][1]),image_height

  horiz_boxes.append([x_h,y_h,x_h+width_h,y_h+height_h])
  vert_boxes.append([x_v,y_v,x_v+width_v,y_v+height_v])

  cv2.rectangle(im,(x_h,y_h), (x_h+width_h,y_h+height_h),(0,0,255),1)
  cv2.rectangle(im,(x_v,y_v), (x_v+width_v,y_v+height_v),(0,255,0),1)


In [77]:
cv2.imwrite('horiz_vert.jpg',im)

True

## Non-Max Suppression

In [79]:
horiz_out = tf.image.non_max_suppression(
    horiz_boxes,
    probabilities,
    max_output_size = 1000,
    iou_threshold=0.1,
    score_threshold=float('-inf'),
    name=None
)

InvalidArgumentError: ignored

In [48]:
horiz_lines = np.sort(np.array(horiz_out))
print(horiz_lines)

[5]


In [49]:
im_nms = image_cv.copy()

In [50]:
for val in horiz_lines:
  cv2.rectangle(im_nms, (int(horiz_boxes[val][0]),int(horiz_boxes[val][1])), (int(horiz_boxes[val][2]),int(horiz_boxes[val][3])),(0,0,255),1)


In [51]:
cv2.imwrite('im_nms.jpg',im_nms)

True

In [52]:
vert_out = tf.image.non_max_suppression(
    vert_boxes,
    probabilities,
    max_output_size = 1000,
    iou_threshold=0.1,
    score_threshold=float('-inf'),
    name=None
)

In [53]:
print(vert_out)

tf.Tensor([5 3 1 0 2 4], shape=(6,), dtype=int32)


In [54]:
vert_lines = np.sort(np.array(vert_out))
print(vert_lines)

[0 1 2 3 4 5]


In [55]:
for val in vert_lines:
  cv2.rectangle(im_nms, (int(vert_boxes[val][0]),int(vert_boxes[val][1])), (int(vert_boxes[val][2]),int(vert_boxes[val][3])),(255,0,0),1)


In [56]:
cv2.imwrite('im_nms.jpg',im_nms)

True

## Convert to CSV

In [57]:


out_array = [["" for i in range(len(vert_lines))] for j in range(len(horiz_lines))]
print(np.array(out_array).shape)
print(out_array)

(1, 6)
[['', '', '', '', '', '']]


In [58]:

unordered_boxes = []

for i in vert_lines:
  print(vert_boxes[i])
  unordered_boxes.append(vert_boxes[i][0])

[7, 0, 103, 21]
[104, 0, 179, 21]
[180, 0, 216, 21]
[220, 0, 295, 21]
[299, 0, 443, 21]
[447, 0, 566, 21]


In [59]:
ordered_boxes = np.argsort(unordered_boxes)
print(ordered_boxes)

[0 1 2 3 4 5]


In [60]:
def intersection(box_1, box_2):
  return [box_2[0], box_1[1],box_2[2], box_1[3]]

In [61]:
def iou(box_1, box_2):

  x_1 = max(box_1[0], box_2[0])
  y_1 = max(box_1[1], box_2[1])
  x_2 = min(box_1[2], box_2[2])
  y_2 = min(box_1[3], box_2[3])

  inter = abs(max((x_2 - x_1, 0)) * max((y_2 - y_1), 0))
  if inter == 0:
      return 0

  box_1_area = abs((box_1[2] - box_1[0]) * (box_1[3] - box_1[1]))
  box_2_area = abs((box_2[2] - box_2[0]) * (box_2[3] - box_2[1]))

  return inter / float(box_1_area + box_2_area - inter)

In [62]:
for i in range(len(horiz_lines)):
  for j in range(len(vert_lines)):
    resultant = intersection(horiz_boxes[horiz_lines[i]], vert_boxes[vert_lines[ordered_boxes[j]]] )

    for b in range(len(boxes)):
      the_box = [boxes[b][0][0],boxes[b][0][1],boxes[b][2][0],boxes[b][2][1]]
      if(iou(resultant,the_box)>0.1):
        out_array[i][j] = texts[b]

In [63]:
out_array=np.array(out_array)

In [64]:
out_array

array([['исходные', 'ае', 'ля', 'расчеа', 'кШИчEИ', 'оказателей']],
      dtype='<U10')

In [65]:
pd.DataFrame(out_array).to_csv('sample.csv')

## Merging Cells

In [47]:
current_bank=['']*len(out_array[0,:])
print(current_bank)

['', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


In [48]:
def empty(arr):
  for i in arr:
    if i=='':
      return True
  return False

In [49]:
cleaned_array=[]

In [50]:
for i in range(len(out_array)):
  if not empty(out_array[i]):
    current_bank=[out_array[i][j] for j in range(len(out_array[i]))]
    cleaned_array.append(current_bank)
    not_empty=True
  else:
    for j in range(len(out_array[i])):
      current_bank[j]+=' '+out_array[i][j]
    print('-->',current_bank)
cleaned_array=np.array(cleaned_array)
print(cleaned_array)

--> [' No I', ' ', ' ', ' ', ' ', ' ', ' оказађели', ' оказађели', ' ', ' ', ' ', ' Ed.', ' измерения', ' начения', ' нароектныйгод']
--> [' No I ', '  ', '  Hефт', '  знтренне', '  знтренне', '  ', ' оказађели ', ' оказађели ', '  ', '  ', '  ', ' Ed. ', ' измерения Pyo.т', ' начения ', ' нароектныйгод зђ']
--> [' No I  ', '   ', '  Hефт Hефты', '  знтренне BHеUHем', '  знтренне BHеUHем', '   ', ' оказађели  ', ' оказађели  ', '   ', '   ', '   ', ' Ed.  доллбаpp.', ' измерения Pyo.т доллбаpp.', ' начения  ', ' нароектныйгод зђ 5O,']
--> [' No I   ', '    ', '  Hефт Hефты ', '  знтренне BHеUHем ', '  знтренне BHеUHем Bнтреннемрынке', '    Bнтреннемрынке', ' оказађели   ', ' оказађели   ', '    ', '    ', '    ', ' Ed.  доллбаpp. ', ' измерения Pyo.т доллбаpp. Pyo.т', ' начения   ', ' нароектныйгод зђ 5O, ']
--> [' No I    ', '     ', '  Hефт Hефты  природныйгаз', '  знтренне BHеUHем  природныйгаз', '  знтренне BHеUHем Bнтреннемрынке ', '    Bнтреннемрынке ', ' оказађели    ', ' оказађ

In [52]:
pd.DataFrame(cleaned_array).to_csv('cleaned.csv')

# Convert to OWL Format

# **CSV to Text**

In [ ]:
import  jpype
import  asposecells


jpype.startJVM()
from asposecells.api import Workbook

workbook = Workbook("/content/sample.csv")
workbook.save("Output.docx")
jpype.shutdownJVM()

ModuleNotFoundError: ignored

# Brighten Image (For Anyone dealing with PDFs created from scanned images)

In [ ]:
from PIL import Image, ImageEnhance

#read the image
im = Image.open("ext_im.jpg")

#image brightness enhancer
enhancer = ImageEnhance.Brightness(im)

factor = 1 #gives original image
im_output = enhancer.enhance(factor)
im_output.save('ext_im-1.jpg')

factor = 1.5## brightens the image
im_output = enhancer.enhance(factor)
im_output.save('ext_im-2.jpg')


In [1]:
!pip install easyocr

In [2]:
import easyocr

In [4]:
def text_recognition(file_path):
    reader = easyocr.Reader(['ru'])
    result = reader.readtext(file_path, detail = 0, paragraph=True)

    return result

def main():
    file_path = input('Insert path:')
    print(text_recognition(file_path=file_path))

if __name__== "__main__":
    main()


Insert path:/content/2023-07-05_15-32-21.png
['Таблица 2.1', 'Состояние запасов растворенного газа Славкинского месторождения', 'Начальые нзнлскасчые ктшпаска мли Утверждениые Тскущне нлкЛскясмыс Накопленная Роснедра " На государстненном злпасы (без учста добыча на (протокол Ха 03-18/493- балансе на 01.01.2018 лобычи 2018) млн; м\' 01,01.2018, пр от 07.092018 г ) МЛН:М А+Н; А+В1 А+В', 'Продуктивные отложення (пласты) объскти, мссторожленнс целом', "Лицснзноннын участок (Липситня €МР 02120 Ю22 Золотарснскос поднятис ПАСТ А-7 Московскин ярус С2ш ПЛАСТ А4 Башкирскии ярус С2Ь ПЛАСТ Б-0 Визсиский ~рус Су ПЛАСТ Б-2 Визснский ~Р€ (у ПЛАСТ В-1 Турнсйский ~.; С1 Итого по Золоваревскому подичтию Сланкинскос 1о [ЛАСТ А-3 Московскни ~рус С2ш ПАСТ А-4 Башкирский ярус €2Ь 6 ПЛАСТ Б-2 Витсйский ярус Су ПЛАСТ Б-3 Витсйский ярус €^у ' Итого ло Славкннскомуполиятню ' Вссто мссторожлсниюь Лнцсшия СИРМ 02120 нэ Нерасираеиныи фомд запасы булут постанлены на госбаланс 01,01,2019 гоца"]
